In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
from alibi_detect.cd import CVMDriftOnline

Importing plotly failed. Interactive plots will not work.


In [2]:
ntimes = 1
times = np.empty([ntimes])

ert = 150  # Desired expected runtime
N = 100 # Size of reference set
B = 10000 # Number of bootstrap simulations for threshold estimation. 100k more realistic?
window_sizes = [10,20]
n_repeats = 200  # Number of instances in stream

# Note how these are diff to the Gaussian used for configuration.
x_ref = np.random.uniform(0, 1, N)
stream_h0 = iter(lambda: np.random.uniform(), 1)
stream_h1 = iter(lambda: np.random.exponential(), 1)

for t in range(ntimes):
    t0 = time.time()
    dd = CVMDriftOnline(x_ref, ert=ert, window_size=window_sizes, n_bootstraps=B, verbose=True, device='parallel')
    times[t] = time.time() - t0

print('Wall time = %.3fs +- %.2gs' % (times.mean(),times.std()))

Using 10000 boostrap simulations to configure thresholds...
Threshold config time = 0.556
Wall time = 0.557s +- 0s


In [16]:
n = 0
while n < 50:
    a = np.array([np.random.uniform()])
    #a = np.array([np.random.randn()])
    print(dd.predict(a)['data']['is_drift'])
    n += 1

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [13]:
np.random.uniform(0,1.1,1).shape

(1,)